In [1]:
import sys
import re
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from optparse import OptionParser
import pandas as pd
import numpy as np
from rdkit.Chem.Descriptors import MolWt, HeavyAtomCount,MolLogP, RingCount, NumAromaticRings, NumHDonors, NumHAcceptors
from rdkit.Chem.Descriptors import NumAromaticRings, qed
from rdkit.Chem.QED import QEDproperties
from rdkit.Chem import rdMMPA
from indexing_call_as_function import cansmirk,add_context_to_smirks,index_from_smiles_re,gen_queries,gen_queries_for_H_transform,add_context_to_lhs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from molvs import standardize_smiles, Standardizer
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.SaltRemover import SaltRemover

import numpy as np
import pandas as pd
import psycopg2
import hidden
import itertools
import time

In [2]:
# Load the secrets
secrets = hidden.secrets()

conn = psycopg2.connect(host=secrets['host'],
        port=secrets['port'],
        database=secrets['database'],
        user=secrets['user'],
        password=secrets['pass'],
        connect_timeout=1000)

cur = conn.cursor()

In [3]:
sql = 'DROP TABLE IF EXISTS ikenacomp_images CASCADE;'
print(sql)
cur.execute(sql)

DROP TABLE IF EXISTS ikenacomp_images CASCADE;


In [4]:
sql = 'CREATE TABLE ikenacomp_images (id SERIAL, Molecule_id varchar(128), '\
      'img bytea, '\
     'primary key(id));'
print(sql)
cur.execute(sql)

CREATE TABLE ikenacomp_images (id SERIAL, Molecule_id varchar(128), img bytea, primary key(id));


In [5]:
suppl = Chem.SDMolSupplier('../MMP_files/CDD_18_April_2022.sdf')

In [6]:
ms = [x for x in suppl if x is not None]
names = [x.GetProp('Molecule Name') for x in ms]
#  name = mol.GetProp('Molecule Name')

In [7]:
len(ms)

8397

In [8]:
len(names)

8397

In [9]:
for i in range (10):
    Draw.MolsToGridImage([ms[0]], subImgSize=(400,400))
    

In [10]:
namesall = []
for x in ms:
    img_name = x.GetProp('Molecule Name')+".png"
    if(img_name not in namesall):
        y = [x]
        img=Draw.MolsToGridImage(y,molsPerRow=1,subImgSize=(400,400),legends=[u.GetProp("Molecule Name") for u in y], returnPNG=False)
        img_ = "C:\MAMP\htdocs\MMP_v3\images\{}".format(img_name)
        img.save(img_)
        namesall.append(img_name)
    

In [11]:
len(namesall)

1693

In [12]:
files = ['../MMP_files/Clearance.csv',
        '../MMP_files/Chembl_t_half.csv',
        '../MMP_files/Bioavail_All_Chembl.csv',
        '../MMP_files/Chembl_herg.csv',
        '../MMP_files/Vdss_All_Chembl.csv',
        ]

In [13]:
df_1 = pd.read_csv('../MMP_files/Clearance.csv')
df_2 = pd.read_csv('../MMP_files/Chembl_t_half.csv')
df_3 = pd.read_csv('../MMP_files/Bioavail_All_Chembl.csv')
df_4 = pd.read_csv('../MMP_files/Chembl_herg.csv')
df_5 = pd.read_csv('../MMP_files/Vdss_All_Chembl.csv')

In [14]:
df = pd.concat([df_1,df_2,df_3,df_3,df_5])

In [15]:
df.columns

Index(['Unnamed: 0', 'assay_chembl_id', 'assay_description',
       'canonical_smiles', 'molecule_chembl_id', 'standard_type',
       'standard_units', 'standard_value', 'type', 'units', 'value',
       'Unnamed: 0.1'],
      dtype='object')

In [16]:
df = df.dropna(subset = ['canonical_smiles'])
df = df.drop_duplicates(subset=['molecule_chembl_id'])

In [17]:
# namesall = []
# for x in ms:
#     img_name = x.GetProp('Molecule Name')+".png"
#     if(img_name not in namesall):
#         y = [x]
#         img=Draw.MolsToGridImage(y,molsPerRow=1,subImgSize=(400,400),legends=[u.GetProp("Molecule Name") for u in y], returnPNG=False)
#         img_ = "C:\MAMP\htdocs\Web_Design_4e\JSEvents\images\{}".format(img_name)
#         img.save(img_)
#         namesall.append(img_name)

In [19]:
namesall = []
for _, x in df.iterrows():
    mol = Chem.MolFromSmiles(x['canonical_smiles'])
    name = x['molecule_chembl_id']
    img_name = name+".png"
    img=Draw.MolsToGridImage([mol],molsPerRow=1,subImgSize=(400,400),legends=[name], returnPNG=False)
    img_ = "C:\MAMP\htdocs\MMP_v3\images\{}".format(img_name)
    img.save(img_)
    namesall.append(img_name)
#     print(img_name)

In [ ]:
# sql = 'select * from ikenacomp_images where ' \
#           'values (%s,%s);'

In [ ]:
conn.commit()
cur.close()